In [1]:
import os
import shutil
import json
import pykakasi
import re
import bisect
import sqlite3
import math
import pickle

In [2]:
katakana = ''.join(map(chr, range(0x30a1, 0x30fb))) + 'ー' + '々'
hiragana = ''.join(map(chr, range(0x3041, 0x3097)))
cjkranges = [
    # compatibility ideographs
    {"from": ord(u"\u3300"), "to": ord(u"\u33ff")},
    # compatibility ideographs
    {"from": ord(u"\ufe30"), "to": ord(u"\ufe4f")},
    # compatibility ideographs
    {"from": ord(u"\uf900"), "to": ord(u"\ufaff")},
    # compatibility ideographs
    {"from": ord(u"\U0002F800"), "to": ord(u"\U0002fa1f")},
    # cjk radicals supplement
    {"from": ord(u"\u2e80"), "to": ord(u"\u2eff")},
    {"from": ord(u"\u3400"), "to": ord(u"\u9fff")},
    {"from": ord(u"\U00020000"), "to": ord(u"\U0002a6df")},
    {"from": ord(u"\U0002a700"), "to": ord(u"\U0002b73f")},
    {"from": ord(u"\U0002b740"), "to": ord(u"\U0002b81f")},
    {"from": ord(u"\U0002b820"), "to": ord(u"\U0002ceaf")},
    {"from": ord(u"\U0002ceb0"), "to": ord(u"\U0002ebef")},
    {"from": ord(u"\U0002f800"), "to": ord(u"\U0002fa1f")},
    {"from": ord(u"\U00030000"), "to": ord(u"\U0003134f")},
]


def is_cjk(char):
    return any([range["from"] <= ord(char) <= range["to"] for range in cjkranges])


latin = range(0x41, 0x17f)
kks = pykakasi.kakasi()

In [3]:
use_cache = False
if os.path.exists('words.pkl'):
    words = pickle.load(open('words.pkl', 'rb'))
    use_cache = True

freqs = {}
for dir in os.listdir('jisho'):
    if 'Freq' in dir:
        index = json.load(open('jisho/'+dir+'/index.json', 'r'))
        for j in filter(lambda x: 'term_meta_bank' in x, os.listdir('jisho/'+dir)):
            for term in json.load(open('jisho/'+dir+'/'+j, 'r')):
                weight = 1 if 'weight' not in index else index['weight']
                word, _, freq = term
                assert _ == 'freq'
                wk = word
                if type(freq) is dict and 'reading' in freq:
                    yomikata = freq['reading']
                    if not all(map(lambda x: x in hiragana, yomikata)):
                        yomikata = ''.join(
                            map(lambda x: x['hira'], kks.convert(yomikata)))
                    wk = (word, yomikata)
                    weight *= 3
                    freq = freq['frequency']
                elif all(map(lambda x: x in hiragana or x in katakana, word)):
                    weight *= 1.8
                if type(freq) is dict and 'value' in freq:
                    if not all(map(lambda x: x in hiragana or x in katakana, word)):
                        if "㋕" in freq['displayValue']:
                            weight *= 0.5
                    freq = freq['value']
                if use_cache:
                    if type(wk) is str and (not all(map(lambda x: x in hiragana or x in katakana, wk))) and wk in words and len(words[wk]) == 1:
                        wk = (wk, words[wk][0][1])
                        weight *= 3
                if wk not in freqs:
                    freqs[wk] = 0
                freqs[wk] += 2**(12 - freq**0.5/30)*weight
        print(
            f"Freq, {index['title']}, {1 if 'weight' not in index else index['weight']}")

Freq, BCCWJ-LUW, 4
Freq, CC100, 4
Freq, Innocent Ranked, 0.01
Freq, JPDB, 0.2
Freq, Netflix, 0.2
Freq, Novels, 0.1
Freq, TWC, 0.6
Freq, Wikipedia, 1
Freq, 国語辞典, 0.8


In [4]:
def parse_struct(x):
    if type(x) is str:
        yield x
    elif x['tag'] == 'img':
        pass
    elif x['tag'] == 'div':
        yield from parse_struct_meta(x['content'])
        yield '\n'
    elif x['tag'] == 'span':
        yield '('
        yield from parse_struct_meta(x['content'])
        yield ')'
    elif x['tag'] == 'a':
        assert type(x['content']) is str
        yield f"【{x['content']}】"
    elif x['tag'] == 'br':
        yield '\n'
    elif x['tag'] == 'ruby':
        yield f"({x['content'][0]})"
    elif x['tag'] == 'table':
        yield from parse_struct_meta(x['content'])
    elif x['tag'] == 'tr':
        yield from parse_struct_meta(x['content'])
        yield '\n'
    elif x['tag'] == 'th' or x['tag'] == 'td':
        yield from parse_struct_meta(x['content'])
        yield ' | '
    else:
        print(x)
        assert 0


def parse_struct_meta(imistruct):
    if type(imistruct) is str:
        yield imistruct
    elif type(imistruct) is dict:
        yield from parse_struct(imistruct)
    elif type(imistruct) is list:
        for x in imistruct:
            yield from parse_struct(x)
    else:
        print(imistruct)
        assert 0

In [5]:
jpdc = {}
dangling = {}
for dir in os.listdir('jisho'):
    if 'lingual' in dir:
        index = json.load(open('jisho/'+dir+'/index.json', 'r'))
        for j in filter(lambda x: 'term_bank' in x, os.listdir('jisho/'+dir)):
            for term in json.load(open('jisho/'+dir+'/'+j, 'r')):
                word, yomikata, _, _, weight, imi, _, _ = term
                if '{{' in word:
                    continue
                if all(map(lambda x: x in katakana or x == '・', word)):
                    word = word.replace('・', '')
                if all(map(lambda x: ord(x) in latin, word)) and len(word) > 1:
                    if yomikata != '' and all(map(lambda x: x in katakana, yomikata)):
                        word = yomikata
                if yomikata == '':
                    yomikata = word
                if all(map(lambda x: x in hiragana, word)) and word != yomikata:
                    continue
                if len(imi) > 1:
                    assert imi[1]['type'] == 'image'
                if not type(imi[0]) is str:
                    assert imi[0]['type'] == 'structured-content'
                    imi[0] = imi[0]['content'][0]+'\n' + \
                        ''.join(parse_struct_meta(imi[0]['content'][1:]))
                if weight < 0:
                    weight = weight/8
                imilen = len(imi[0]) - len(word)
                imilen = (imilen - (len(yomikata) if yomikata !=
                          word else 0)) if imilen > len(yomikata) else imilen
                weight += math.sqrt(imilen) / \
                    (200 if index['title'] == 'JMdict' else 100)
                if not all(map(lambda x: x in katakana or x in hiragana, yomikata)):
                    currentdc = dangling
                    currentkey = word
                else:
                    yomikata = ''.join(
                        map(lambda x: x['hira'], kks.convert(yomikata)))
                    currentdc = jpdc
                    currentkey = (word, yomikata)
                if currentkey in currentdc:
                    if index['title'] not in currentdc[currentkey][1]:
                        currentdc[currentkey][1][index['title']] = []
                    if imi[0] not in currentdc[currentkey][1][index['title']]:
                        currentdc[currentkey][1][index['title']].append(imi[0])
                        currentdc[currentkey][0] += weight + 5
                else:
                    currentdc[currentkey] = [weight, {index['title']:[imi[0]]}]
        print(f"Lingual, {index['title']}")

Lingual, JMdict
Lingual, 新世纪日汉双解大辞典
Lingual, Weblio 古語辞典
Lingual, デジタル大辞泉
Lingual, ハイブリッド新辞林
Lingual, 三省堂 必携類語実用辞典
Lingual, 大辞林 第三版
Lingual, 学研 四字熟語辞典
Lingual, 実用日本語表現辞典
Lingual, 岩波国語辞典 第六版
Lingual, 広辞苑 第七版
Lingual, 故事ことわざの辞典
Lingual, 新明解四字熟語辞典
Lingual, 新明解国語辞典 第五版
Lingual, どんなときどう使う 日本語表現文型辞典
Lingual, 旺文社国語辞典 第十一版 画像無し
Lingual, 明鏡国語辞典 第二版
Lingual, 語源由来辞典


In [6]:
# Try to fix incorrectly merged/splitted entries
toRemove = set()
toAdd = {}
for word, yomikata in jpdc:
    if '・' in word:
        flag = "UND"
        for w in word.split('・'):
            if (w, yomikata) in jpdc:
                if "デジタル大辞泉" in jpdc[(w, yomikata)][1]:
                    flag = "OK"
                elif "デジタル大辞泉" in jpdc[(word, yomikata)][1]:
                    if flag == "UND":
                        flag = "BAD"
        if flag == "BAD":
            for w in word.split('・'):
                if (w, yomikata) in jpdc:
                    temp = jpdc[(w, yomikata)]
                    toRemove.add((w, yomikata))
                    jpdc[(word, yomikata)][0] += temp[0]
                    for k, v in temp[1].items():
                        if k not in jpdc[(word, yomikata)][1]:
                            jpdc[(word, yomikata)][1][k] = v
                        else:
                            for vv in v:
                                if vv not in jpdc[(word, yomikata)][1][k]:
                                    jpdc[(word, yomikata)][1][k].append(vv)
        elif flag == "OK":
            temp = jpdc[(word, yomikata)]
            toRemove.add((word, yomikata))
            for w in word.split('・'):
                if (w, yomikata) in jpdc:
                    jpdc[(w, yomikata)][0] += temp[0]
                    for k, v in temp[1].items():
                        if k not in jpdc[(w, yomikata)][1]:
                            jpdc[(w, yomikata)][1][k] = v
                        else:
                            for vv in v:
                                if vv not in jpdc[(word, yomikata)][1][k]:
                                    jpdc[(word, yomikata)][1][k].append(vv)
                else:
                    toAdd[(w, yomikata)] = temp
for k in toRemove:
    del jpdc[k]
for k in toAdd:
    jpdc[k] = toAdd[k].copy()

for term in [('ごいちろくぐんじくーでたー', ['五', '一六軍事クーデター']), ('によんでぃー', ['2']), ('よんいちくがくせいかくめい', ['四', '一九学生革命']), ('にいにいろくじけん', ['二', '二六事件']), ('ににはちじけん', ['二', '二八事件']), ('ぴーえむにてんご', ['5', 'PM2'])]:
    yomi, ws = term
    for w in ws:
        del jpdc[(w, yomi)]

In [7]:
words = {}
yomikatas = {}
for word, yomikata in jpdc:
    if word not in words:
        words[word] = [[jpdc[(word, yomikata)][0], yomikata]]
    else:
        words[word].append([jpdc[(word, yomikata)][0], yomikata])
    if yomikata not in yomikatas:
        yomikatas[yomikata] = [[jpdc[(word, yomikata)][0], word]]
    else:
        yomikatas[yomikata].append([jpdc[(word, yomikata)][0], word])

In [8]:
# Fix dangling entries
for w in dangling:
    if w in words:
        if len(words[w]) == 1 or len(w) == 1:
            for yw in words[w]:
                keys = (w, yw[1])
                jpdc[keys][0] += dangling[w][0]
                yw[0] += dangling[w][0]
                for t in yomikatas[keys[1]]:
                    if t[1] == w:
                        t[0] += dangling[w][0]
                for books in dangling[w][1]:
                    if books not in jpdc[keys][1]:
                        jpdc[keys][1][books] = []
                    for imis in dangling[w][1][books]:
                        if imis not in jpdc[keys][1][books]:
                            jpdc[keys][1][books].append(imis)

In [9]:
for word, yomikata in jpdc:
    if word in freqs:
        jpdc[(word, yomikata)][0] += freqs[word]
    if (word, yomikata) in freqs:
        jpdc[(word, yomikata)][0] += freqs[(word, yomikata)]

In [10]:
for word in words:
    words[word] = sorted(words[word], key=lambda x: x[0], reverse=True)
for yomikata in yomikatas:
    yomikatas[yomikata] = sorted(
        yomikatas[yomikata], key=lambda x: x[0], reverse=True)

In [11]:
# try to merge same term
for yomikata in yomikatas:
    for i in range(len(yomikatas[yomikata])):
        wordi = yomikatas[yomikata][i][1]
        if all(map(lambda x: x in katakana or x in hiragana, wordi)) and len(yomikatas[yomikata]) > 2:
            continue
        imis = jpdc[(wordi, yomikata)][1]
        for j in range(i+1, len(yomikatas[yomikata])):
            wordj = yomikatas[yomikata][j][1]
            if all(map(lambda x: x in katakana or x in hiragana, wordj)) and len(yomikatas[yomikata]) > 2:
                continue
            imisj = jpdc[(wordj, yomikata)][1]
            flag = False
            for k in set(imis.keys()).intersection(set(imisj.keys())):
                flag = True
                if imis[k] != imisj[k]:
                    flag = False
                    break
            if flag:
                for k in imisj:
                    imis[k] = imisj[k]
                jpdc[(wordj, yomikata)][1] = imis

In [12]:
# Manual one-way merge some frequently used pairs
oneway_pairs = [('する', '為る'), ('など', '等'), ('まで', '迄'), ('について', 'に就いて'), ('ではない', 'では無い'), ('ほど', '程'), ('てしまう', 'て仕舞う'), ('ながら', '乍ら'), ('そして', '然して'), ('によって', 'に因って'), ('による', 'に依る'), ('べし', '可し'), ('ことができる', '事ができる'), ('ことになる', '事になる'), ('つく', '付く'), ('ばかり', '許り'), ('にとって', 'に取って'), ('かもしれない', 'かも知れない'), ('どうして', '如何して'), ('といった', 'と言った'), ('ことがある', '事がある'), ('どうしても', '如何しても'), ('ことにする', '事にする'), ('ことはない', '事はない'), ('あんた', '貴方'), ('によると', 'に依ると'), ('こっち', '此方'), ('をもって', 'を以て'), ('とる', '取る'), ('によれば', 'に依れば'), ('さっき', '先'), ('とはいえ', 'とは言え'), ('といっても', 'と言っても'), ('にわたって', 'に渡って'), ('こととなる', '事となる'), ('あちこち', '彼方此方'), ('なる', '成る'), ('までもない', 'までも無い'), ('による', 'に依る'), ('にわたり', 'に渡り'), ('ここ', '此処')]
for yomi in yomikatas:
    if len(yomikatas[yomi]) == 2:
        if yomikatas[yomi][0][1] == yomi:
            if any(map(is_cjk, yomikatas[yomi][1][1])):
                oneway_pairs.append((yomi,yomikatas[yomi][1][1]))
        if yomikatas[yomi][1][1] == yomi:
            if any(map(is_cjk, yomikatas[yomi][0][1])):
                oneway_pairs.append((yomi,yomikatas[yomi][0][1]))

In [13]:
for pair in oneway_pairs:
    for k in jpdc[(pair[1],pair[0])][1]:
        if k not in jpdc[(pair[0], pair[0])][1]:
            jpdc[(pair[0], pair[0])][1][k] = []
        for imi in jpdc[(pair[1],pair[0])][1][k]:
            if imi not in jpdc[(pair[0], pair[0])][1][k]:
                jpdc[(pair[0], pair[0])][1][k].append(imi)

In [14]:
weightlist = list(map(lambda x: x[0], jpdc.values()))
weightlist.sort()
for w in jpdc:
    jpdc[w].append(len(weightlist)-bisect.bisect_right(weightlist, jpdc[w][0]))

In [15]:
for dir in os.listdir('jisho/'):
    if 'Pitch Accent' in dir:
        index = json.load(open('jisho/'+dir+'/index.json', 'r'))
        for j in filter(lambda x: 'term_meta_bank' in x, os.listdir('jisho/'+dir)):
            for term in json.load(open('jisho/'+dir+'/'+j, 'r')):
                word, _, pitch = term
                assert _ == 'pitch'
                yomi = ''.join(
                    map(lambda x: x['hira'], kks.convert(pitch['reading'])))
                if (word, yomi) in jpdc:
                    if len(jpdc[(word, yomi)]) == 3:
                        jpdc[(word, yomi)].append(set())
                    for p in pitch['pitches']:
                        jpdc[(word, yomi)][3].add(p['position'])
        print(f"Pitches, {index['title']}")

Pitches, Kanjium Pitch Accents
Pitches, アクセント辞典


In [16]:
pickle.dump(jpdc, open('jpdc.pkl', 'wb'))
pickle.dump(yomikatas, open('yomikatas.pkl', 'wb'))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(freqs, open('freqs.pkl', 'wb'))

In [17]:
#jpdc = pickle.load(open('jpdc.pkl', 'rb'))
#yomikatas = pickle.load(open('yomikatas.pkl', 'rb'))
#words = pickle.load(open('words.pkl', 'rb'))
#freqs = pickle.load(open('freqs.pkl', 'rb'))

In [18]:
cw = {}
for w, yomi in jpdc:
    for c in w:
        if is_cjk(c):
            if c not in cw:
                cw[c] = 0
            cw[c] += jpdc[(w, yomi)][0]

cwth = sorted(cw.items(), key=lambda x: x[1], reverse=True)[2600][1]

In [19]:
univar = []
for l in list(filter(lambda x: len(x) > 4 and x[0] != '#', open('unihan/Unihan_Variants.txt').read().split('\n'))):
    l = l.split('\t')
    if l[1] == 'kSpecializedSemanticVariant':
        continue
    a = chr(int(l[0][2:], 16))
    for b in l[2].split(' '):
        b = chr(int(b.split('<')[0][2:], 16))
        assert is_cjk(a) and is_cjk(b)
        univar.append((a, b))

In [20]:
shinjitai = "亜（亞） 悪（惡） 圧（壓） 囲（圍） 医（醫） 為（爲） 壱（壹） 逸（逸） 隠（隱） 栄（榮） 営（營） 衛（衞） 駅（驛） 謁（謁） 円（圓） 塩（鹽） 縁（緣） 艶（艷） 応（應） 欧（歐） 殴（毆） 桜（櫻） 奥（奧） 横（橫） 温（溫） 穏（穩） 仮（假） 価（價） 禍（禍） 画（畫） 会（會） 悔（悔） 海（海） 絵（繪） 壊（壞） 懐（懷） 慨（慨） 概（槪） 拡（擴） 殻（殼） 覚（覺） 学（學） 岳（嶽） 楽（樂） 喝（喝） 渇（渴） 褐（褐） 缶（罐） 巻（卷） 陥（陷） 勧（勸） 寛（寬） 漢（漢） 関（關） 歓（歡） 観（觀） 気（氣） 祈（祈） 既（既） 帰（歸） 亀（龜） 器（器） 偽（僞） 戯（戲） 犠（犧） 旧（舊） 拠（據） 挙（擧） 虚（虛） 峡（峽） 挟（挾） 狭（狹） 郷（鄕） 響（響） 暁（曉） 勤（勤） 謹（謹） 区（區） 駆（驅） 勲（勳） 薫（薰） 径（徑） 茎（莖） 恵（惠） 掲（揭） 渓（溪） 経（經） 蛍（螢） 軽（輕） 継（繼） 鶏（鷄） 芸（藝） 撃（擊） 欠（缺） 研（硏） 県（縣） 倹（儉） 剣（劍） 険（險） 圏（圈） 検（檢） 献（獻） 権（權） 顕（顯） 験（驗） 厳（嚴） 広（廣） 効（效） 恒（恆） 黄（黃） 鉱（鑛） 号（號） 国（國） 黒（黑） 穀（穀） 砕（碎） 済（濟） 斎（齋） 剤（劑） 殺（殺） 雑（雜） 参（參） 桟（棧） 蚕（蠶） 惨（慘） 賛（贊） 残（殘） 糸（絲） 祉（祉） 視（視） 歯（齒） 児（兒） 辞（辭） 湿（濕） 実（實） 写（寫） 社（社） 者（者） 煮（煮） 釈（釋） 寿（壽） 収（收） 臭（臭） 従（從） 渋（澁） 獣（獸） 縦（縱） 祝（祝） 粛（肅） 処（處） 暑（暑） 署（署） 緒（緖） 諸（諸） 叙（敍） 将（將） 祥（祥） 称（稱） 渉（涉） 焼（燒） 証（證） 奨（奬） 条（條） 状（狀） 乗（乘） 浄（淨） 剰（剩） 畳（疊） 縄（繩） 壌（壤） 嬢（孃） 譲（讓） 醸（釀） 触（觸） 嘱（囑） 神（神） 真（眞） 寝（寢） 慎（愼） 尽（盡） 図（圖） 粋（粹） 酔（醉） 穂（穗） 随（隨） 髄（髓） 枢（樞） 数（數） 瀬（瀨） 声（聲） 斉（齊） 静（靜） 窃（竊） 摂（攝） 節（節） 専（專） 浅（淺） 戦（戰） 践（踐） 銭（錢） 潜（潛） 繊（纖） 禅（禪） 祖（祖） 双（雙） 壮（壯） 争（爭） 荘（莊） 捜（搜） 挿（插） 巣（巢） 曽（曾） 痩（瘦） 装（裝） 僧（僧） 層（層） 総（總） 騒（騷） 増（增） 憎（憎） 蔵（藏） 贈（贈） 臓（臟） 即（卽） 属（屬） 続（續） 堕（墮） 対（對） 体（體） 帯（帶） 滞（滯） 台（臺） 滝（瀧） 択（擇） 沢（澤） 担（擔） 単（單） 胆（膽） 嘆（嘆） 団（團） 断（斷） 弾（彈） 遅（遲） 痴（癡） 虫（蟲） 昼（晝） 鋳（鑄） 著（著） 庁（廳） 徴（徵） 聴（聽） 懲（懲） 勅（敕） 鎮（鎭） 塚（塚） 逓（遞） 鉄（鐵） 点（點） 転（轉） 伝（傳） 都（都） 灯（燈） 当（當） 党（黨） 盗（盜） 稲（稻） 闘（鬭） 徳（德） 独（獨） 読（讀） 突（突） 届（屆） 難（難） 弐（貳） 悩（惱） 脳（腦） 覇（霸） 拝（拜） 廃（廢） 売（賣） 梅（梅） 麦（麥） 発（發） 髪（髮） 抜（拔） 繁（繁） 晩（晚） 蛮（蠻） 卑（卑） 秘（祕） 碑（碑） 浜（濱） 賓（賓） 頻（頻） 敏（敏） 瓶（甁） 侮（侮） 福（福） 払（拂） 仏（佛） 併（倂） 並（竝） 塀（塀） 餅（餠） 辺（邊） 変（變） 弁（辨） 弁（瓣） 弁（辯） 勉（勉） 歩（步） 宝（寶） 豊（豐） 褒（襃） 墨（墨） 翻（飜） 毎（每） 万（萬） 満（滿） 免（免） 麺（麵） 黙（默） 弥（彌） 訳（譯） 薬（藥） 与（與） 予（豫） 余（餘） 誉（譽） 揺（搖） 様（樣） 謡（謠） 来（來） 頼（賴） 乱（亂） 覧（覽） 欄（欄） 竜（龍） 隆（隆） 虜（虜） 両（兩） 猟（獵） 緑（綠） 涙（淚） 塁（壘） 類（類） 礼（禮） 励（勵） 戻（戾） 霊（靈） 齢（齡） 暦（曆） 歴（歷） 恋（戀） 練（練） 錬（鍊） 炉（爐） 労（勞） 郎（郞） 朗（朗） 廊（廊） 楼（樓） 録（錄） 湾（灣） 亘（亙） 凜（凛） 尭（堯） 巌（巖） 晃（晄） 桧（檜） 槙（槇） 渚（渚） 猪（豬） 琢（琢） 祢（禰） 祐（祐） 祷（禱） 禄（祿） 禎（禎） 穣（穰） 萌（萠） 遥（遙） 唖（啞） 頴（穎） 鴎（鷗） 撹（攪） 麹（麴） 鹸（鹼） 噛（嚙） 繍（繡） 蒋（蔣） 醤（醬） 曽（曾） 掻（搔） 痩（瘦） 祷（禱） 屏（屛） 并（幷） 桝（枡） 麺（麵） 沪（濾） 芦（蘆） 蝋（蠟） 弯（彎）"
for l in shinjitai.split(' '):
    a = l[0]
    b = l[2]
    assert is_cjk(a) and is_cjk(b)
    assert a != b
    univar.append((a, b))

In [21]:
for txt in os.listdir('sts-data'):
    with open('sts-data/'+txt, 'r') as f:
        for l in f.read().split('\n'):
            l = l.split('\t')
            if len(l) > 1:
                a = l[0]
                for b in l[1].split(' '):
                    univar.append((a, b))

In [22]:
class DisjointSet(object):

    def __init__(self, weight):
        self.leader = {}  # maps a member to the group's leader
        self.group = {}  # maps a group leader to the group (which is a set)
        self.weight = weight

    def comp(self, leadera, leaderb, groupa, groupb):
        if leadera in self.weight:
            if leaderb in self.weight:
                return self.weight[leadera] < self.weight[leaderb]
            else:
                return False
        elif leaderb in self.weight:
            return True
        else:
            return len(groupa) < len(groupb)

    def add(self, a, b):
        if a not in self.leader:
            self.leader[a] = a
            self.group[a] = set([a])
        if b not in self.leader:
            self.leader[b] = b
            self.group[b] = set([b])
        leadera = self.leader.get(a)
        leaderb = self.leader.get(b)
        if leadera == leaderb:
            return  # nothing to do
        groupa = self.group[leadera]
        groupb = self.group[leaderb]
        if self.comp(leadera, leaderb, groupa, groupb):
            a, leadera, groupa, b, leaderb, groupb = b, leaderb, groupb, a, leadera, groupa
        groupa |= groupb
        del self.group[leaderb]
        for k in groupb:
            self.leader[k] = leadera


ds = DisjointSet(cw)
for a, b in univar:
    ds.add(a, b)

In [23]:
for x in ds.group.copy():
    if x not in cw:
        del ds.group[x]
for x in ds.leader.copy():
    if ds.leader[x] not in cw:
        del ds.leader[x]

In [24]:
cjconvert = {}
for x in ds.leader:
    if (x not in cw or cw[x] < cwth) and x != ds.leader[x]:
        cjconvert[x] = ds.leader[x]

In [25]:
convertjs = json.dumps(cjconvert, ensure_ascii=False)
with open('../js/cjconvert.js', 'w') as f:
    f.write(f"const cjdc = {convertjs};\n")
    f.write(
        "function cj_convert(s) {return s.split('').map(c=>cjdc[c]||c).join('');}")

In [26]:
ml = 0
for w in words:
    if any(map(lambda x: x in cjconvert, w)):
        if ml < len(w):
            ml = len(w)
assert max(map(len, words.keys())) < 100 and max(
    map(len, yomikatas.keys())) < 60 and ml < 40

In [27]:
con = sqlite3.connect("../db/arujisho.db")
cur = con.cursor()

In [28]:
cur.execute('''CREATE VIRTUAL TABLE IF NOT EXISTS jpdc USING fts4(word NVARCHAR(100) NOT NULL, yomikata NVARCHAR(60) NOT NULL, romaji VARCHAR(108) NOT NULL,
            freqRank INTEGER NOT NULL NOT INDEXED,
            rword NVARCHAR(100) NOT NULL, ryomikata NVARCHAR(60) NOT NULL, rromaji VARCHAR(108) NOT NULL,
            pitchData VARCHAR(20) NOT INDEXED,
            origForm NVARCHAR(40) NOT INDEXED,
            idex INTEGER NOT NULL)''')
cur.execute(
    '''CREATE TABLE IF NOT EXISTS imis (imi TEXT NOT NULL, orig TEXT NOT NULL)''')

In [29]:
dups = {}
c = 1
for w, v in sorted(jpdc.items(), key=lambda x: x[1][2]):
    jData = json.dumps(v[1], ensure_ascii=False, sort_keys=True)
    if jData not in dups:
        cur.execute('INSERT INTO imis VALUES (?, ?)', (jData, w[0]))
        dups[jData] = (c, w[0])
        c += 1
    romaji = ''.join(
        map(lambda x: x['hepburn'], kks.convert(w[1]))).replace("'", '')
    word = ''.join(
        map(lambda x: x if x not in cjconvert else cjconvert[x], w[0]))
    cur.execute('INSERT INTO jpdc VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (word, w[1], romaji, v[2],
                 word[::-1], w[1][::-1], romaji[::-1],
                 str(sorted(list(v[3]))) if len(v) > 3 else '', '' if word == w[0] else w[0], dups[jData][0]))

In [30]:
con.commit()
con.close()